In [1]:
import pandas as pd
import numpy as np
from scripts.models.parameters import Parameters
import scripts.main as main
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import networkx as nx
import math
import random

In [2]:
generate = True
num_agents = 10

# 500
productivities = [50] * num_agents
num_steps = 100

pars = Parameters()
pars.n = num_agents

# Generate Erdős–Rényi graph
probability = 0.8
#G = nx.erdos_renyi_graph(num_agents, probability)

# Generate k-regular graph
k = 8
G = nx.random_regular_graph(k, num_agents)
    
Adj = nx.to_numpy_array(G)
print(Adj)
W = Adj / 2
pars.b = W

pars.neib_add = 5
pars.add_neib_val = 0.5
pars.static_system = True

r = 0.5
pr = max([productivities[agent] for agent in range(num_agents)])
#L = 0.505
#mu = 0.9
L = 0.505
mu = 0.79
#L = math.sqrt((num_agents + 1) * max([(r ** 2) / (productivities[agent] ** 4) for agent in range(num_agents)]))
#mu = min([np.sum(string) / (100 ** 2) for string in Adj])
h = 0.002
eta = 0.8
alpha = 0.003
gamma = [0.007]
#alpha = 0.001
#gamma = [0.001]
#alpha = 0.00003
#gamma = [0.00007]

pars.params_dict = {
    "L": L,
    "mu": mu,
    "h": h,
    "eta": eta,
    "gamma": gamma,
    "alpha": alpha
}

print(pars.params_dict)

[[0. 1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 0. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 0. 1. 1. 1.]
 [1. 1. 0. 1. 0. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 0. 1. 1. 1. 1.]
 [1. 1. 1. 0. 1. 1. 0. 1. 1. 1.]
 [1. 0. 1. 1. 1. 1. 1. 0. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]
 [1. 1. 1. 1. 1. 1. 1. 1. 0. 0.]]
{'L': 0.505, 'mu': 0.79, 'h': 0.002, 'eta': 0.8, 'gamma': [0.007], 'alpha': 0.003}


In [3]:
exp = 1
iter_past = 0

def add_error(df, column_name, num):
    df["Mean"] = df.mean(axis=1)
    for i in range(num):
        df[f"{column_name}_{i}_sqr"] = np.power(df[f"{column_name}_{i}"] - df[f"Mean"], 2)
    df["Error"] = np.sqrt(sum([df[f"{column_name}_{i}_sqr"] for i in range(num)])/num)
    return df

def add_worst_case_error(df, column_name, num):
    df["Mean"] = df.mean(axis=1)
    for i in range(num):
        df[f"{column_name}_{i}_value"] = np.abs(df[f"{column_name}_{i}"] - df[f"Mean"])
    df["Worst-case-error"] = np.max([df[f"{column_name}_{i}_value"] for i in range(num)])
    return df

def plot(seq_df, num_agents, alg):
    """ Plot queue length dynamics """
    global k, exp
    
    fig = make_subplots(rows=2, cols=1)

    for i in range(num_agents):
        sequence = seq_df[f"Agent_{i}"]
        fig.add_trace(go.Scatter(y=sequence,          
                                 x=list(range(len(sequence))),
                                             name=f"Agent {i} {alg}",
                                             mode="lines"),
                     row=1, col=1)

    fig.update_layout(
        xaxis_title="t",
        yaxis_title="Queue length",
        font = dict(size = 20),
        height=600,
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.show()

    fig.write_html(f"./trash/trash_{k}_{exp}_{alg}_seq.html")

random_task_generate = [random.randint(0, 100) for _ in range(100)]
def run(is_evaluate, algorithm, num_agents, num_steps, pars, generate, productivities, ret_alg=False):
    global k, exp
    
    alg = None
    if is_evaluate:
        print(algorithm)
        alg = main.run_load_balancing(algorithm, num_steps, pars, generate, random_task_generate, productivities=productivities)
    
        print(algorithm)
        alg_seq = alg.sequence
        seq_df = pd.DataFrame(alg_seq, columns=[f"Agent_{ind}" for ind in range(num_agents)])
        
        plot(seq_df, num_agents, alg=algorithm)
        
        seq_df = add_error(seq_df, "Agent", num_agents)
        seq_df = add_worst_case_error(seq_df, "Agent", num_agents)
        seq_df.to_csv(f"./trash/seq/{exp}_{k}_{algorithm}_seq.html")
    else:
        seq_df = pd.read_csv(f"./trash/seq/{exp}_{k}_{algorithm}_seq.html")
        print(f"Added seq_df")
    
    if ret_alg:
        return alg, seq_df
    return None, seq_df

In [4]:
itr = 1
start = -1
iter_past = 0

for k in range(iter_past, itr):
    # Run LVP and extract sequnce
    do = k > start
    alg_lvp, lvp_seq = run(do, main.LVP, num_agents, num_steps, pars, True, productivities, ret_alg=True) 

    # Run ALVP
    do = k > start
    alg_alvp, alvp_seq = run(do, main.ALVP, num_agents, num_steps, pars, False, productivities, ret_alg=True) 

    # Run RR
    do = k > start
    alg_rr, rr_seq = run(do, main.ROUND_ROBIN, num_agents, num_steps, pars, False, productivities, ret_alg=True) 
    
    # Plot error plot
    layout = go.Layout(plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(layout=layout)

    fig.add_trace(go.Scatter(y=lvp_seq["Error"],
                                         x=list(range(len(lvp_seq))),
                                         name=f"Error LVP",
                                         mode="lines",
                            line={'dash': 'dot', 'color': 'red'}))

    fig.add_trace(go.Scatter(y=alvp_seq["Error"],
                                 x=list(range(len(alvp_seq))),
                                 name=f"Error ALVP",
                                 mode="lines",
                            line={'dash': 'solid', 'color': 'green'}))


    fig.add_trace(go.Scatter(y=rr_seq["Error"],
                                 x=list(range(len(rr_seq))),
                                 name=f"Error RR",
                                 mode="lines",
                            line={'dash': 'solid', 'color': 'blue'}))


    fig.update_layout(
        xaxis_title="t",
        yaxis_title="Error",
        legend = dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.9
        ),
        font = dict(size = 20),
        height=800
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.show()

    fig.write_html(f"./trash/trash_{k}_{exp}_error.html");   

    # Plot error plot
    layout = go.Layout(plot_bgcolor='rgba(0,0,0,0)')
    fig = go.Figure(layout=layout)

    fig.add_trace(go.Scatter(y=lvp_seq["Worst-case-error"],
                                         x=list(range(len(lvp_seq))),
                                         name=f"Worst-case-error LVP",
                                         mode="lines",
                            line={'dash': 'dot', 'color': 'red'}))

    fig.add_trace(go.Scatter(y=alvp_seq["Worst-case-error"],
                                 x=list(range(len(alvp_seq))),
                                 name=f"Worst-case-error ALVP",
                                 mode="lines",
                            line={'dash': 'solid', 'color': 'green'}))


    fig.add_trace(go.Scatter(y=rr_seq["Worst-case-error"],
                                 x=list(range(len(rr_seq))),
                                 name=f"Worst-case-error RR",
                                 mode="lines",
                            line={'dash': 'solid', 'color': 'blue'}))


    fig.update_layout(
        xaxis_title="t",
        yaxis_title="Worst-case-error",
        legend = dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.9
        ),
        font = dict(size = 20),
        height=800
    )
    fig.update_xaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.update_yaxes(showline=True, linewidth=2, linecolor='black', gridcolor='white')
    fig.show()

    fig.write_html(f"./trash/trash_{k}_{exp}_worst_case_error.html");     
    
print("Finish")
iter_past = k

LVP
Step 0 is completed
Step 1 is completed
Step 2 is completed
Step 3 is completed
Step 4 is completed
Step 5 is completed
Step 6 is completed
Step 7 is completed
Step 8 is completed
Step 9 is completed
Step 10 is completed
Step 11 is completed
2
Step 12 is completed
Step 13 is completed
Step 14 is completed
Step 15 is completed
Step 16 is completed
Step 17 is completed
Step 18 is completed
Step 19 is completed
Step 20 is completed
Step 21 is completed
Step 22 is completed
Step 23 is completed
Step 24 is completed
Step 25 is completed
Step 26 is completed
Step 27 is completed
1
Step 28 is completed
Step 29 is completed
Step 30 is completed
Step 31 is completed
Step 32 is completed
Step 33 is completed
Step 34 is completed
Step 35 is completed
Step 36 is completed
9
Step 37 is completed
3
Step 38 is completed
Step 39 is completed
Step 40 is completed
Step 41 is completed
Step 42 is completed
Step 43 is completed
Step 44 is completed
7
Step 45 is completed
Step 46 is completed
Step 47 i

ALVP
Step 0 is completed
Step 1 is completed
Step 2 is completed
Step 3 is completed
Step 4 is completed
Step 5 is completed
Step 6 is completed
Step 7 is completed
Step 8 is completed
Step 9 is completed
Step 10 is completed
Step 11 is completed
2
Step 12 is completed
Step 13 is completed
Step 14 is completed
Step 15 is completed
Step 16 is completed
Step 17 is completed
Step 18 is completed
Step 19 is completed
Step 20 is completed
Step 21 is completed
Step 22 is completed
Step 23 is completed
Step 24 is completed
Step 25 is completed
Step 26 is completed
Step 27 is completed
1
Step 28 is completed
Step 29 is completed
Step 30 is completed
Step 31 is completed
Step 32 is completed
Step 33 is completed
Step 34 is completed
Step 35 is completed
Step 36 is completed
9
Step 37 is completed
3
Step 38 is completed
Step 39 is completed
Step 40 is completed
Step 41 is completed
Step 42 is completed
Step 43 is completed
Step 44 is completed
7
Step 45 is completed
Step 46 is completed
Step 47 

ROUND_ROBIN
Step 0 is completed
Step 1 is completed
Step 2 is completed
Step 3 is completed
Step 4 is completed
Step 5 is completed
Step 6 is completed
Step 7 is completed
Step 8 is completed
Step 9 is completed
Step 10 is completed
Step 11 is completed
2
Step 12 is completed
Step 13 is completed
Step 14 is completed
Step 15 is completed
Step 16 is completed
Step 17 is completed
Step 18 is completed
Step 19 is completed
Step 20 is completed
Step 21 is completed
Step 22 is completed
Step 23 is completed
Step 24 is completed
Step 25 is completed
Step 26 is completed
Step 27 is completed
1
Step 28 is completed
Step 29 is completed
Step 30 is completed
Step 31 is completed
Step 32 is completed
Step 33 is completed
Step 34 is completed
Step 35 is completed
Step 36 is completed
9
Step 37 is completed
3
Step 38 is completed
Step 39 is completed
Step 40 is completed
Step 41 is completed
Step 42 is completed
Step 43 is completed
Step 44 is completed
7
Step 45 is completed
Step 46 is completed
S

Finish
